<a href="https://colab.research.google.com/github/lucas-azdias/Programacao-Logica-e-Funcional/blob/main/Formas%20Normais/Formas%20Normais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lucas Azevedo Dias

Obs.: The test text file  must have the elements separated with spaces.

In [1]:
from IPython.display import display, Math
from copy import deepcopy


# Downloading the text file from my GitHub to the virtual machine

In [2]:
import requests
import os


DEFAULT_FOLDER = "files" # Default folder
FILE_NAME = "fbfs" # Filename
FILE_EXT = "txt" # File extension
# Path for the saved text file in my GitHub
PATH = "https://raw.githubusercontent.com/lucas-azdias/Programacao-Logica-e-Funcional/main/Formas%20Normais/"

# Generating the folder path
if os.path.basename(os.getcwd()) != DEFAULT_FOLDER:
  folderPath = os.path.join(os.getcwd(), DEFAULT_FOLDER)
  if not os.path.exists(folderPath):
    os.makedirs(DEFAULT_FOLDER)
  os.chdir(folderPath)

# Downloading file in the given folder
print("\nDownloading files from GitHub...")
filename = FILE_NAME + "." + FILE_EXT
with open(filename, "wb") as file:
  file.write(requests.get(PATH + filename, allow_redirects=True).content)
  file.close()
print(f"{filename} downloaded.")


fbfs.txt downloaded.


# Constants

In [3]:
SEPARATOR = " "  # Separator for the LaTeX texts

## Operators LaTeX texts

In [4]:
EQUIVALENCE = r"\equiv"
BI_IMPLICATION = r"\leftrightarrow"
IMPLICATION = r"\rightarrow"
DISJUNCTION = r"\vee"
CONJUNCTION = r"\wedge"
NEGATION = r"\neg"
FOR_ALL = r"\forall"
EXISTS = r"\exists"

## Enum for each type of operators

In [5]:
BI_OP = 0
MONO_OP = 1
QUANT_OP = 2

## Mapping of operators to their types

In [6]:
OPERATOR_MAPPING = {
    EQUIVALENCE: BI_OP,
    BI_IMPLICATION: BI_OP,
    IMPLICATION: BI_OP,
    DISJUNCTION: BI_OP,
    CONJUNCTION: BI_OP,
    FOR_ALL: QUANT_OP,
    EXISTS: QUANT_OP,
    NEGATION: MONO_OP
}

# Classes

## Base Classes

In [7]:
class LogicElement():
    __symbol = None

    def __init__(self, symbol: str) -> None:
        if not isinstance(symbol, str):
            raise TypeError("Non-string passed.")
        self.__symbol = symbol

    def __str__(self) -> str:
        return self.__symbol.__str__()

    def is_equals(self, x) -> bool:
        return self.__str__() == x.__str__()

    def isValidLogicElement(x: any) -> bool:
        return isinstance(x, LogicElement)

    def verifyLogicElement(*args: any) -> None:
        for arg in args:
            if not LogicElement.isValidLogicElement(arg):
                raise TypeError("One or more invalid logic elements were passed.")


class Variable(LogicElement):
    def __init__(self, symbol: str) -> None:
        super().__init__(symbol)

    def isValidVariable(x: any) -> bool:
        return isinstance(x, Variable)

    def verifyVariable(*args: any) -> None:
        for arg in args:
            if not Variable.isValidVariable(arg):
                raise TypeError("One or more invalid variables were passed.")


class Operator(LogicElement):
    __data = None

    def __init__(self, symbol: str, *args: tuple[str, LogicElement]) -> None:
        super().__init__(symbol)
        self.__data = {}
        for arg in args:
            if isinstance(arg[0], str) and LogicElement.isValidLogicElement(arg[1]):
                self.__data[arg[0]] = arg[1]
            else:
                raise TypeError("One or more invalid arguments were passed.")

    def __getitem__(self, key: str) -> LogicElement:
        return self.__data[key]

    def __setitem__(self, key: str, value: LogicElement) -> None:
        LogicElement.verifyLogicElement(value)
        self.__data[key] = value

    def __iter__(self) -> LogicElement:
        for k, v in self.get_all():
            yield (k, v)

    def is_equals(self, x) -> bool:
        stillEqual = type(self) == type(x)
        if stillEqual:
            for i, e in self:
                if type(e) != type(x[i]):
                    stillEqual = False
                    break
                else:
                    stillEqual = e.is_equals(x[i])
        return stillEqual

    def get_all(self) -> tuple:
        return self.__data.items()

    def solve(self) -> LogicElement:
        return self

    def solve_cnf(self) -> LogicElement:
        return self.solve()

    def solve_dnf(self) -> LogicElement:
        return self.solve()

    def isValidExpression(x: any) -> bool:
        return isinstance(x, Operator)

    def verifyExpression(*args: any) -> None:
        for arg in args:
            if not Operator.isValidExpression(arg):
                raise TypeError("One or more invalid expressions were passed.")

## Bi-arguments branch

In [8]:
class BiOperator(Operator):
    def __init__(self, a: LogicElement, b: LogicElement, symbol: str) -> None:
        super().__init__(symbol, ("a", a), ("b", b))

    def __str__(self) -> str:
        return "(" + self.get_a().__str__() + SEPARATOR + super().__str__() + SEPARATOR + self.get_b().__str__() + ")"

    def get_a(self) -> LogicElement:
        return self.__getitem__("a")

    def get_b(self) -> LogicElement:
        return self.__getitem__("b")


class Equivalence(BiOperator):
    def __init__(self, a: LogicElement, b: LogicElement) -> None:
        super().__init__(a, b, EQUIVALENCE)

    def solve(self) -> LogicElement:  # Equivalence
        a = self.get_a()
        b = self.get_b()
        return BiImplication(a, b)


class BiImplication(BiOperator):
    def __init__(self, a: LogicElement, b: LogicElement) -> None:
        super().__init__(a, b, BI_IMPLICATION)

    def solve(self) -> LogicElement:  # Biimplication
        a = self.get_a()
        b = self.get_b()
        return Conjunction(Implication(a, b), Implication(b, a))


class Implication(BiOperator):
    def __init__(self, a: LogicElement, b: LogicElement) -> None:
        super().__init__(a, b, IMPLICATION)

    def solve(self) -> LogicElement:  # Rule of inference
        a = self.get_a()
        b = self.get_b()
        return Disjunction(Not(a), b)


class Disjunction(BiOperator):
    def __init__(self, a: LogicElement, b: LogicElement) -> None:
        super().__init__(a, b, DISJUNCTION)

    def solve(self) -> LogicElement:  # Simplification
        a = self.get_a()
        b = self.get_b()
        if a.is_equals(b):
            return a
        return self

    def solve_cnf(self) -> LogicElement:  # Distribution
        operator = self.solve()
        if isinstance(operator, Disjunction):
            a = operator.get_a()
            b = operator.get_b()
            if isinstance(a, Conjunction):
                a_a = a.get_a()
                b_a = a.get_b()
                return Conjunction(Disjunction(b, a_a), Disjunction(b, b_a))
            elif isinstance(b, Conjunction):
                a_b = b.get_a()
                b_b = b.get_b()
                return Conjunction(Disjunction(a, a_b), Disjunction(a, b_b))
        return operator


class Conjunction(BiOperator):
    def __init__(self, a: LogicElement, b: LogicElement) -> None:
        super().__init__(a, b, CONJUNCTION)

    def solve(self) -> LogicElement:  # Simplification
        a = self.get_a()
        b = self.get_b()
        if a.is_equals(b):
            return a
        return self

    def solve_dnf(self) -> LogicElement:  # Distribution
        operator = self.solve()
        if isinstance(operator, Conjunction):
            a = operator.get_a()
            b = operator.get_b()
            if isinstance(a, Disjunction):
                a_a = a.get_a()
                b_a = a.get_b()
                return Disjunction(Conjunction(b, a_a), Conjunction(b, b_a))
            elif isinstance(b, Disjunction):
                a_b = b.get_a()
                b_b = b.get_b()
                return Disjunction(Conjunction(a, a_b), Conjunction(a, b_b))
        return operator

## Mono-arguments branch

In [9]:
class MonoOperator(Operator):
    def __init__(self, a: LogicElement, symbol: str) -> None:
        super().__init__(symbol, ("a", a))

    def __str__(self) -> str:
        return super().__str__() + SEPARATOR + self.get_a().__str__()

    def get_a(self) -> LogicElement:
        return self.__getitem__("a")


class Not(MonoOperator):
    def __init__(self, a: LogicElement) -> None:
        super().__init__(a, NEGATION)

    def solve(self) -> LogicElement:
        a = self.get_a()
        if isinstance(a, Disjunction):  # Morgan's Law
            a_a = a.get_a()
            b_a = a.get_b()
            return Conjunction(Not(a_a), Not(b_a))
        elif isinstance(a, Conjunction):  # Morgan's Law
            a_a = a.get_a()
            b_a = a.get_b()
            return Disjunction(Not(a_a), Not(b_a))
        elif isinstance(a, Not):  # Double negation
            a_a = a.get_a()
            return a_a
        else:
            return self

## Quantifiers branch

In [10]:
class Quantifier(Operator):
    def __init__(self, var: LogicElement, expr: LogicElement, symbol: str) -> None:
        Variable.verifyVariable(var)
        super().__init__(symbol, ("var", var), ("expr", expr))

    def __str__(self) -> str:
        return "(" + super().__str__() + SEPARATOR + self.get_var().__str__() + SEPARATOR + "(" + self.get_expr().__str__() + ")" + ")"

    def get_var(self) -> LogicElement:
        return self.__getitem__("var")

    def get_expr(self) -> LogicElement:
        return self.__getitem__("expr")


class ForAll(Quantifier):
    def __init__(self, var: LogicElement, expr: LogicElement) -> None:
        super().__init__(var, expr, FOR_ALL)


class Exists(Quantifier):
    def __init__(self, var: LogicElement, expr: LogicElement) -> None:
        super().__init__(var, expr, EXISTS)

# Functions

## Basic functions

In [11]:
def print_latex(latex_text: str) -> None:
    # Prints the given LaTeX text as such
    display(Math(latex_text))

## Parsing functions

In [12]:
def parse_latex(latex_text: str) -> LogicElement:
    # Parses the LaTeX text into a LogicElement according to the rules of precedence (recursively)

    latex_text = latex_text.strip()

    splitted = split_latex(latex_text)  # Splits the LaTeX text into individual elements

    selected_op = select_operation(splitted)  # Decides the operation based on precedence rules

    # Parses the data considering the operation selected (recursively)
    if selected_op:
        index = splitted.index(selected_op)
        if OPERATOR_MAPPING[selected_op] == BI_OP:
            a = parse_sub_elements(splitted[:index])
            b = parse_sub_elements(splitted[index + 1:])

            if selected_op == EQUIVALENCE:
                return Equivalence(a, b)
            elif selected_op == BI_IMPLICATION:
                return BiImplication(a, b)
            elif selected_op == IMPLICATION:
                return Implication(a, b)
            elif selected_op == DISJUNCTION:
                return Disjunction(a, b)
            elif selected_op == CONJUNCTION:
                return Conjunction(a, b)
        elif OPERATOR_MAPPING[selected_op] == MONO_OP:
            a = parse_sub_elements(splitted[index + 1])

            if selected_op == NEGATION:
                return Not(a)
        elif OPERATOR_MAPPING[selected_op] == QUANT_OP:
            a = parse_sub_elements(splitted[index + 1])
            b = parse_sub_elements(splitted[index + 2:])

            if selected_op == FOR_ALL:
                return ForAll(a, b)
            elif selected_op == EXISTS:
                return Exists(a, b)

    return Variable(SEPARATOR.join([str(e) for e in splitted]))  # Default case when no operator is selected


def parse_sub_elements(elements: list[str] | str) -> LogicElement:
    # Returns the sub elements given from an expression parsed
    text = SEPARATOR.join([str(e) for e in elements])
    if text[0] == "(" and text[-1] == ")":  # Removes brackets if it has
        text = text[1:-1]
    return parse_latex(text)


def split_latex(latex_text: str) -> list:
    # Splits the LaTeX text into individual elements
    from queue import LifoQueue

    # Verifies brackets
    if latex_text.count("(") != latex_text.count(")"):
        raise ValueError("Invalid LaTeX expression passed.")

    splitted = []  # List for elements of the expression

    brackets_stack = LifoQueue()  # Stack to counter brackets
    buffer = ""  # Buffer for letters between brackets
    for i, c in enumerate(latex_text):  # Separates elements according to brackets
        if c == SEPARATOR and brackets_stack.empty():
            splitted.append(buffer)
            buffer = ""
        else:
            if c == "(":
                brackets_stack.put(c)
            elif c == ")":
                brackets_stack.get()
            buffer += c
    splitted.append(buffer)

    return splitted


def select_operation(latex_splitted: list) -> str:
    # Selects the operation based on precedence rules (and returns the exact operation as string)

    selected = None  # Default case when no operator is found

    for operator in OPERATOR_MAPPING:
        if operator in latex_splitted:
            selected = operator
            break

    # Decides the quantifier based in order (if it has both)
    if selected and OPERATOR_MAPPING[selected] == QUANT_OP and (FOR_ALL in latex_splitted and EXISTS in latex_splitted):
            fa_index = latex_splitted.index(FOR_ALL) if FOR_ALL in latex_splitted else len(latex_splitted)
            ex_index = latex_splitted.index(EXISTS) if EXISTS in latex_splitted else len(latex_splitted)

            selected = FOR_ALL if fa_index < ex_index else EXISTS

    return selected

# Solving functions

## Conjunction normal form (CNF) solver

In [13]:
def cnf_solver(parsed_latex: LogicElement) -> LogicElement:
    # Solve the CNF for the parsed LaTeX and returns it (recursively)

    parsed = deepcopy(parsed_latex)  # Copies the instance and all its nested objects

    # Checks if it's an expression (otherwise it skips)
    if isinstance(parsed, Operator):
        last_parsed = None

        # If there is any changes, it tries to solve more, otherwise it's finished
        while parsed != last_parsed:

            last_parsed = parsed  # Takes the new base for comparison

            # Sends the solver message to inner
            if isinstance(parsed, Operator):
                for i, e in parsed:
                    parsed[i] = cnf_solver(e)

                # Solves CNF of expression
                parsed = parsed.solve_cnf()

    return parsed

## Disjunction normal form (DNF) solver

In [14]:
def dnf_solver(parsed_latex: LogicElement) -> LogicElement:
    # Solve the DNF for the parsed LaTeX and returns it

    parsed = deepcopy(parsed_latex)  # Copies the instance and all its nested objects

    # Checks if it's an expression (otherwise it skips)
    if isinstance(parsed, Operator):
        last_parsed = None

        # If there is any changes, it tries to solve more, otherwise it's finished
        while parsed != last_parsed:

            last_parsed = parsed  # Takes the new base for comparison

            # Sends the solver message to inner
            if isinstance(parsed, Operator):
                for i, e in parsed:
                    parsed[i] = dnf_solver(e)

                # Solves DNF of expression
                parsed = parsed.solve_dnf()

    return parsed

# Main code

In [15]:
if __name__ == "__main__":
    # Initial messages
    print("\033[34mCNF/DNF SOLVER\n")
    print("\033[34mProgram started...")

    # Reading file
    print("\033[34mReading file... ", end="")
    file_lines = []
    with open(filename, "r") as file:
        for line in file.readlines():
            file_lines.append(line.removesuffix("\n"))
    print("\033[34mFile read successfully.")

    # Solving CNF/DNF for each line
    print("\033[34m\nSolving CNF/DNF for each line...\n")
    for i, line in enumerate(file_lines):
        print(f"\033[33mSolving line {i + 1} (\033[32m\"{line}\"\033[33m)...")
        print("\033[33mParsing latex text... ", end="")
        parsed_latex = parse_latex(line)
        cnf = str(cnf_solver(parsed_latex))
        dnf = str(dnf_solver(parsed_latex))
        print("\033[33mLatex text parsed successfully.")
        print(f"\033[33mParsed: (\033[32m\"{parsed_latex}\"\033[33m)", end="")
        print_latex(line)
        print(f"\033[33mCNF: (\033[32m\"{cnf}\"\033[33m)", end="")
        print_latex(cnf)
        print(f"\033[33mDNF: (\033[32m\"{dnf}\"\033[33m)", end="")
        print_latex(dnf)
        print()
    print("\033[34mEvery line has been solved successfully.\n")

    # End message
    print("\033[34mProgram finished.")

CNF/DNF SOLVER

Program started...
Reading file... File read successfully.

Solving CNF/DNF for each line...

Solving line 1 ("A \rightarrow B \vee C")...
Parsing latex text... Latex text parsed successfully.
Parsed: ("(A \rightarrow (B \vee C))")

<IPython.core.display.Math object>

CNF: ("(\neg A \vee (B \vee C))")

<IPython.core.display.Math object>

DNF: ("(\neg A \vee (B \vee C))")

...

CNF: ("((\neg (\forall A (P(A))) \vee B) \wedge (\neg B \vee (\forall A (P(A)))))")

<IPython.core.display.Math object>

DNF: ("(((\neg (\forall A (P(A))) \wedge \neg B) \vee (\neg (\forall A (P(A))) \wedge (\forall A (P(A))))) \vee ((B \wedge \neg B) \vee (B \wedge (\forall A (P(A))))))")

<IPython.core.display.Math object>


Every line has been solved successfully.

Program finished.
